In [3]:
# Setup
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
import time
import warnings
import torch.nn as nn
import pickle
from dataset_loader import *
from utils import *
import pandas as pd
from datasets import Dataset, DatasetDict

SEED = 84
torch.manual_seed(SEED)
warnings.simplefilter("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
dataset_name = 'dilemma'
ds = get_ds(dataset_name)

In [124]:
#model_name ='google/electra-small-discriminator'
model_name =  'huawei-noah/TinyBERT_General_4L_312D'
folder_name = 'tinybert'

In [129]:
set_seed()
model = load_model(model_name)
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}', num_train_epochs=10)
model.save_pretrained(f'{folder_name}/{dataset_name}/model')

loading configuration file config.json from cache at /home/almr16/.cache/huggingface/hub/models--huawei-noah--TinyBERT_General_4L_312D/snapshots/34707a33cd59a94ecde241ac209bf35103691b43/config.json
Model config BertConfig {
  "_name_or_path": "huawei-noah/TinyBERT_General_4L_312D",
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "structure": [],
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/almr16/.cache/huggingface/hub/models--huawei-noah--TinyBERT_General_4L_312

  0%|          | 0/6074 [00:00<?, ?ex/s]

  0%|          | 0/1072 [00:00<?, ?ex/s]

PyTorch: setting up devices
***** Running training *****
  Num examples = 6074
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1900


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.318385,0.877799
2,No log,0.247738,0.911381
3,0.325700,0.211977,0.918843
4,0.325700,0.261067,0.914179
5,0.325700,0.315021,0.908582
6,0.140000,0.264322,0.931903
7,0.140000,0.271779,0.929104
8,0.089400,0.344877,0.917910
9,0.089400,0.329266,0.923507
10,0.089400,0.330501,0.926306


***** Running Evaluation *****
  Num examples = 1072
  Batch size = 32
Saving model checkpoint to tinybert/dilemma/checkpoint-190
Configuration saved in tinybert/dilemma/checkpoint-190/config.json
Model weights saved in tinybert/dilemma/checkpoint-190/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1072
  Batch size = 32
Saving model checkpoint to tinybert/dilemma/checkpoint-380
Configuration saved in tinybert/dilemma/checkpoint-380/config.json
Model weights saved in tinybert/dilemma/checkpoint-380/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1072
  Batch size = 32
Saving model checkpoint to tinybert/dilemma/checkpoint-570
Configuration saved in tinybert/dilemma/checkpoint-570/config.json
Model weights saved in tinybert/dilemma/checkpoint-570/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 1072
  Batch size = 32
Saving model checkpoint to tinybert/dilemma/checkpoint-760
Configuration saved in tinybert/dilemma/checkpoint-760/confi

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [29]:
def predict_sentences(sentences):
    encoded = [[101] +[tokenizer._convert_token_to_id_with_added_voc(token) for token in tokens] + [102]         
               for tokens in sentences]
    to_pred = torch.tensor(encoded, device=device)
    outputs = traced_model(to_pred)[0]
    #print(outputs)
    return torch.argmax(outputs, dim=1).cpu().numpy()

### save best model's folder as 'model'

In [136]:
model = AutoModelForSequenceClassification.from_pretrained(f'{folder_name}/{dataset_name}/model', num_labels=2, torchscript=True).to(device)
model = model.eval()

loading configuration file tinybert/dilemma/model/config.json
Model config BertConfig {
  "_name_or_path": "tinybert/dilemma/model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "problem_type": "single_label_classification",
  "structure": [],
  "torch_dtype": "float32",
  "torchscript": true,
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file tinybert/dilemma/model/pytorch_model.bin
All model checkpoint weights were used when initializing Be

In [94]:
tokenized_data = tokenize_dataset(ds, tokenizer_name=model_name, max_length = 64)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /home/almr16/.cache/huggingface/hub/models--huawei-noah--TinyBERT_General_4L_312D/snapshots/34707a33cd59a94ecde241ac209bf35103691b43/config.json
Model config BertConfig {
  "_name_or_path": "huawei-noah/TinyBERT_General_4L_312D",
  "attention_probs_dropout_prob": 0.1,
  "cell": {},
  "classifier_dropout": null,
  "emb_size": 312,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "initializer_range": 0.02,
  "intermediate_size": 1200,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 4,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "pre_trained": "",
  "structure": [],
  "transformers_version": "4.23.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file vocab.txt from cache at /ho

  0%|          | 0/8000 [00:00<?, ?ex/s]

  0%|          | 0/2000 [00:00<?, ?ex/s]

In [77]:
train(model, tokenized_data, path=f'{folder_name}/{dataset_name}', evaluate = True)

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 2087
  Batch size = 32


{'eval_loss': 0.22128722071647644,
 'eval_accuracy': 0.9195016770483948,
 'eval_runtime': 0.7969,
 'eval_samples_per_second': 2618.843,
 'eval_steps_per_second': 82.819}

In [137]:
encoded = tokenizer.encode("i love movies", add_special_tokens=True, return_tensors="pt").to(device)
traced_model = torch.jit.trace(model, encoded)

In [138]:
torch.jit.save(traced_model, f"{folder_name}/{dataset_name}/traced.pt")

In [10]:
traced_model = torch.jit.load(f"{folder_name}/{dataset_name}/traced.pt")

In [11]:
predict_sentences([tokenizer.tokenize("i hate movies")])

tensor([[ 1.9854, -1.9343]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([0])

In [12]:
predict_sentences([tokenizer.tokenize("i love movies")])

tensor([[-1.9767,  1.9226]], device='cuda:0', grad_fn=<AddmmBackward0>)


array([1])

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad
count_parameters(model)

In [ ]:
# check accuracy per class

positives = ds['test'].filter(lambda x: x['label'] == 1)
negatives = ds['test'].filter(lambda x: x['label'] == 0)

pos_ds = DatasetDict()
pos_ds['train'] = positives 
pos_ds['test'] = positives
pos_data = tokenize_dataset(pos_ds, tokenizer_name = model_name, max_length = 64)

neg_ds = DatasetDict()
neg_ds['train'] = negatives 
neg_ds['test'] = negatives
neg_data = tokenize_dataset(neg_ds, tokenizer_name = model_name, max_length = 64)

In [80]:
# sentiment
train(model, pos_data, path=f'', evaluate = True)

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 4163
  Batch size = 32


{'eval_loss': 0.22361314296722412,
 'eval_accuracy': 0.9092000960845544,
 'eval_runtime': 1.5854,
 'eval_samples_per_second': 2625.779,
 'eval_steps_per_second': 82.627}

In [81]:
# sentiment
train(model, neg_data, path=f'', evaluate = True)

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 4183
  Batch size = 32


{'eval_loss': 0.2286013960838318,
 'eval_accuracy': 0.9261295720774564,
 'eval_runtime': 1.5588,
 'eval_samples_per_second': 2683.468,
 'eval_steps_per_second': 84.039}

In [96]:
# corona
train(model, pos_data, path=f'', evaluate = True)

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 1031
  Batch size = 32


{'eval_loss': 0.25200405716896057,
 'eval_accuracy': 0.895247332686712,
 'eval_runtime': 0.4264,
 'eval_samples_per_second': 2417.668,
 'eval_steps_per_second': 77.384}

In [97]:
# corona
train(model, neg_data, path=f'', evaluate = True)

PyTorch: setting up devices
***** Running Evaluation *****
  Num examples = 969
  Batch size = 32


{'eval_loss': 0.3279201090335846,
 'eval_accuracy': 0.8710010319917441,
 'eval_runtime': 0.3809,
 'eval_samples_per_second': 2543.924,
 'eval_steps_per_second': 81.385}